In [1]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from helpers import *

In [2]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

print(gt_imgs[0].shape)
print(imgs[0].shape)

Loading 100 images
Loading 100 images
(400, 400)
(400, 400, 3)


In [3]:
def img_crop_v2(im, w, h, sw, sh):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight-h+1,sh):
        for j in range(0,imgwidth-w+1,sw):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

def pad_image(image, padding):
    if (len(image.shape) < 3): # gt image
        data = np.pad(image, ((padding, padding), (padding, padding)), 'reflect')
    else:
        data = np.pad(image, ((padding, padding), (padding, padding), (0, 0)), 'reflect')
    return data

# Extract patches from input images
window_size = 72
patch_size = 16 # each patch is 16*16 pixels
padding_size = (window_size - patch_size) // 2

img_patches = [img_crop_v2(pad_image(imgs[i], padding_size), window_size, window_size, patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop_v2(pad_image(gt_imgs[i], padding_size), window_size, window_size, patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

print(img_patches.shape)
print(gt_patches.shape)

(62500, 72, 72, 3)
(62500, 72, 72)


In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ReLU, Dropout, Flatten, Dense, ELU, Activation
from keras.regularizers import l2

INPUT_SHAPE=(window_size, window_size, 3)
REG = 1e-6

# Use Max Pool or a Conv2D Layer with increased stride
useMaxPool = True
useReLU = True

model = Sequential()
model.add(Conv2D(filters=128,
                 kernel_size=(5, 5),
                 strides=(1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
if useMaxPool:
    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=None,
                           padding='same'))
else:
    model.add(Conv2D(filters = 128,
                     kernel_size = (5, 5),
                     strides = (2, 2),
                     padding='same',
                     input_shape=INPUT_SHAPE))
#model.add(Dropout(0.25))
model.add(Conv2D(filters = 256,
                 kernel_size = (5, 5),
                 strides = (1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
#model.add(ReLU(max_value=2))
if useMaxPool:
    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=None,
                           padding='same'))
else:
    model.add(Conv2D(filters = 256,
                     kernel_size = (4, 4),
                     strides = (2, 2),
                     padding='same',
                     input_shape=INPUT_SHAPE))
#model.add(Dropout(0.25))
model.add(Conv2D(filters = 512,
                 kernel_size = (4, 4),
                 strides = (1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
#model.add(ReLU(max_value=2))
if useMaxPool:
    model.add(MaxPooling2D(pool_size=(2, 2),
                           strides=None,
                           padding='same'))
else:
    model.add(Conv2D(filters = 512,
                     kernel_size = (4, 4),
                     strides = (2, 2),
                     padding='same',
                     input_shape=INPUT_SHAPE))
model.add(Flatten())
model.add(Dense(1024, kernel_regularizer=l2(REG)))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
#model.add(ReLU(max_value=2))
model.add(Dropout(0.25))
model.add(Dense(512, kernel_regularizer=l2(REG)))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
model.add(Dropout(0.5))
model.add(Dense(128, kernel_regularizer=l2(REG)))
if useReLU:
    model.add(ReLU())
else:
    model.add(ELU())
#model.add(ReLU(max_value=2))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_regularizer=l2(REG)))
model.add(Activation('softmax'))

print(model.summary())

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 72, 72, 128)       9728      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 72, 72, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 256)       819456    
_________________________________________________________________
re_lu_2 (ReLU)               (None, 36, 36, 256)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 256)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 512)       2097664   
__________

In [5]:
from keras.optimizers import Adam

opt = Adam(lr=0.001) # Adam optimizer with default initial learning rate
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [6]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def gt_to_class(gt_patch):
    v = np.mean(gt_patch[padding_size:padding_size+patch_size, padding_size:padding_size+patch_size])
    df = np.sum(v)
    if df > foreground_threshold:
        return [0, 1]
    else:
        return [1, 0]

X = np.asarray([ img_patches[i] for i in range(len(img_patches))])
Y = np.asarray([gt_to_class(gt_patches[i]) for i in range(len(gt_patches))])
print((Y[:, 0] == 1).sum())
print((Y[:, 1] == 1).sum())

46309
16191


In [7]:
N = X.shape[0]
idx_0 = [idx for idx in range(N) if Y[idx][0] == 1]
idx_1 = [idx for idx in range(N) if Y[idx][1] == 1]

np.random.shuffle(idx_1)
np.random.shuffle(idx_0)
L = min(len(idx_0), len(idx_1))

newX = np.append(X[idx_0[:L]], X[idx_1[:L]], axis=0)
newY = np.append(Y[idx_0[:L]], Y[idx_1[:L]], axis=0)

assert(2 * min((Y[:, 0] == 1).sum(), (Y[:, 1] == 1).sum()) == newX.shape[0])
assert(2 * min((Y[:, 0] == 1).sum(), (Y[:, 1] == 1).sum()) == newY.shape[0])

In [8]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

STEPS_PER_EPOCH = 125
BATCH_SIZE = 250
EPOCHS = 200

datagen = ImageDataGenerator(rotation_range=15,
                             horizontal_flip=True,
                             vertical_flip=True)

datagen.fit(newX)

In [ ]:
from keras.utils import np_utils

lr_callback = ReduceLROnPlateau(monitor='loss',
                                factor=0.2,
                                patience=5,
                                min_lr=0.001)

stop_callback = EarlyStopping(monitor='acc',
                              min_delta=0.0001,
                              patience=11,
                              verbose=1,
                              mode='auto')

checkpoint_callback = ModelCheckpoint('model.h5',
                                      monitor='loss',
                                      verbose=1,
                                      save_best_only=True,
                                      save_weights_only=False,
                                      mode='auto',
                                      period=1)

augment_data = True
batch_size = 125
augment_data = True
samples_per_epoch = X.shape[0]*X.shape[1]*X.shape[2]//256
nb_epoch = 200
nb_classes = 2

def train(X, Y):
    def generate_minibatch():
        while 1:
            # Generate one minibatch
            X_batch = np.empty((batch_size, window_size, window_size, 3))
            Y_batch = np.empty((batch_size, 2))
            for i in range(batch_size):
                # Select a random image
                idx = np.random.choice(X.shape[0])
                shape = X[idx].shape

                # Sample a random window from the image
                center = np.random.randint(window_size//2, shape[0] - window_size//2, 2)
                sub_image = X[idx][center[0]-window_size//2:center[0]+window_size//2,
                                   center[1]-window_size//2:center[1]+window_size//2]
                gt_sub_image = Y[idx][center[0]-patch_size//2:center[0]+patch_size//2,
                                      center[1]-patch_size//2:center[1]+patch_size//2]

                # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
                threshold = 0.25
                label = (np.array([np.mean(gt_sub_image)]) > threshold) * 1

                # Image augmentation
                # Random flip
                if np.random.choice(2) == 0:
                    # Flip vertically
                    sub_image = np.flipud(sub_image)
                if np.random.choice(2) == 0:
                    # Flip horizontally
                    sub_image = np.fliplr(sub_image)

                # Random rotation in steps of 90°
                num_rot = np.random.choice(4)
                sub_image = np.rot90(sub_image, num_rot)

                label = np_utils.to_categorical(label, nb_classes)
                X_batch[i] = sub_image
                Y_batch[i] = label

            yield (X_batch, Y_batch)
    if augment_data:
        #model.fit_generator(datagen.flow(X, Y, batch_size=BATCH_SIZE),
        #      steps_per_epoch=X.shape[0] // BATCH_SIZE,
        #      epochs=EPOCHS,
        #      callbacks=[stop_callback, checkpoint_callback],
        #      verbose=1)
        model.fit_generator(generate_minibatch(),
            steps_per_epoch=samples_per_epoch//batch_size//10,
            epochs=nb_epoch,
            verbose=1,
            callbacks=[checkpoint_callback, stop_callback])
    else:
        model.fit(X, Y, callbacks=[stop_callback, checkpoint_callback])

train(np.asarray(imgs), np.asarray(gt_imgs))

Epoch 1/200
1012/1012 [==============================] - 1964s 2s/step - loss: 0.2023 - acc: 0.9184

Epoch 00001: loss improved from inf to 0.20234, saving model to model.h5
Epoch 2/200
1012/1012 [==============================] - 1962s 2s/step - loss: 0.1838 - acc: 0.9269

Epoch 00002: loss improved from 0.20234 to 0.18378, saving model to model.h5
Epoch 3/200
1012/1012 [==============================] - 1974s 2s/step - loss: 0.1708 - acc: 0.9332

Epoch 00003: loss improved from 0.18378 to 0.17078, saving model to model.h5
Epoch 4/200
1012/1012 [==============================] - 1962s 2s/step - loss: 0.1657 - acc: 0.9364

Epoch 00004: loss improved from 0.17078 to 0.16568, saving model to model.h5
Epoch 5/200
 335/1012 [========>.....................] - ETA: 21:39 - loss: 0.1584 - acc: 0.9396

In [9]:
model.load_weights('model.h5')

In [10]:
def predict(X):
    patches = img_crop_v2(pad_image(X, padding_size), window_size, window_size, patch_size, patch_size)
    img_patches = np.zeros(shape=(len(patches), window_size, window_size, 3))
    for index_patch, patch in enumerate(patches):
        img_patches[index_patch] = patch
    Z = model.predict(img_patches)
    Z = (Z[:,0] < Z[:,1]) * 1
    Z = Z.reshape(img_patches.shape[0], -1)
    return Z

def mask_to_submission_strings(model, image_filename):
    """ Reads a single image and outputs the strings that should go into the submission file. """
    img_number = int(re.search(r"\d+", image_filename).group(0))
    Xi = load_image(image_filename)
    Zi = predict(Xi)
    Zi = Zi.reshape(-1)
    patch_size = 16
    nb = 0
    cnt = np.zeros(2)
    print("Processing " + image_filename)
    for j in range(0, Xi.shape[1], patch_size):
        for i in range(0, Xi.shape[0], patch_size):
            label = int(Zi[nb])
            cnt[label] += 1
            nb += 1
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))
    print("cnt[0]=" + str(cnt[0]))
    print("cnt[1]=" + str(cnt[1]))

def generate_submission(model, submission_filename, image_filenames):
    """ Generate a .csv containing the classification of the test set. """
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames:
            f.writelines(['{}\n'.format(s) for s in mask_to_submission_strings(model, fn)])

from time import time
generate_submission(model, f"cnn_v{int(time())}.csv", [f'test_set_images/test_{str(i)}/test_{str(i)}.png' for i in range(1, 51)])

Processing test_set_images/test_1/test_1.png
cnt[0]=971.0
cnt[1]=473.0
Processing test_set_images/test_2/test_2.png
cnt[0]=1138.0
cnt[1]=306.0
Processing test_set_images/test_3/test_3.png
cnt[0]=1095.0
cnt[1]=349.0
Processing test_set_images/test_4/test_4.png
cnt[0]=1181.0
cnt[1]=263.0
Processing test_set_images/test_5/test_5.png
cnt[0]=1097.0
cnt[1]=347.0
Processing test_set_images/test_6/test_6.png
cnt[0]=1076.0
cnt[1]=368.0
Processing test_set_images/test_7/test_7.png
cnt[0]=1106.0
cnt[1]=338.0
Processing test_set_images/test_8/test_8.png
cnt[0]=1070.0
cnt[1]=374.0
Processing test_set_images/test_9/test_9.png
cnt[0]=672.0
cnt[1]=772.0
Processing test_set_images/test_10/test_10.png
cnt[0]=1030.0
cnt[1]=414.0
Processing test_set_images/test_11/test_11.png
cnt[0]=1014.0
cnt[1]=430.0
Processing test_set_images/test_12/test_12.png
cnt[0]=1259.0
cnt[1]=185.0
Processing test_set_images/test_13/test_13.png
cnt[0]=1204.0
cnt[1]=240.0
Processing test_set_images/test_14/test_14.png
cnt[0]=1005